In [33]:
import pandas as pd
import warnings
import re
import tables
import mysql.connector
warnings.filterwarnings('ignore')

In [34]:
def abre_conexao_banco_de_dados():
  global mydb
  mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="4d5f0l9k",
  database="bioinformatica_new"
)

In [35]:
def retorna_tabelas_banco_de_dados(nomeTabela):
  if nomeTabela =='genealogia_embrapa':
    sql = f"WITH tabela_virtual AS (SELECT * FROM genealogia_embrapa) SELECT tv.*, (SELECT registro FROM tabela_virtual AS pai WHERE pai.id_produto = tv.id_touro) AS registro_pai, (SELECT registro FROM tabela_virtual AS mae WHERE mae.id_produto = tv.id_vaca) AS registro_mae FROM tabela_virtual AS tv"
  else:
    sql = f"SELECT * FROM {nomeTabela}"
  df =  pd.read_sql(sql, mydb)
  return df

In [36]:
def cria_df_com_somente_os_dados_novos(planilhaAntiga, planilhaNova):
  dados_em_comum = pd.merge(dados_2023, dados_2022, on=['IDANIMAL', 'AP', 'MP','L305', 'LTOT', 'L365', 'ME', 'AE', 'CAUSA'],suffixes=('', '_drop'))
  dados_em_comum = dados_em_comum.drop(dados_em_comum.filter(regex='_drop$').columns, axis=1)
  dados_em_comum = dados_em_comum.drop_duplicates()
  df_merge = pd.merge(dados_2023, dados_em_comum, how='left', indicator=True, suffixes=('', '_drop'))
  # filtra somente as linhas que estão no dataframe da esquerda
  df_dados_novos = df_merge[df_merge['_merge'] == 'left_only']
  # remove a coluna "_merge"
  df_dados_novos.drop('_merge', axis=1, inplace=True)
  df_dados_novos = df_dados_novos.drop(df_merge.filter(regex='_drop$').columns, axis=1)
  df_dados_novos = df_dados_novos.drop_duplicates()
  return df_dados_novos

In [37]:
def adiciona_coluna_gs_aos_dados_novos(df,df_pedigree ):
  df_temporaria = pd.merge(df, df_pedigree[['IDANIMAL', 'gs', 'SEXO']], on=['IDANIMAL'], suffixes=('', '_drop'))
  df=df_temporaria
  nome_antigo = 'SEXO'
  nome_novo = 'sexo'
  indice_coluna = df.columns.get_loc(nome_antigo)
  df.rename(columns={nome_antigo: nome_novo}, inplace=True)
  return df

In [38]:
def retira_acentos_espacos(df):
  # remoção dos hifens e espaços em branco da coluna RGVACA
  df.loc[df['COMPRAC'] == 11, 'RGVACA'] = df.loc[df['COMPRAC'] == 11, 'RGVACA'].str.replace('-', '').str.replace(' ', '')

  df.loc[df['GSMAE'] == 11, 'RGMAE'] = df.loc[df['GSMAE'] == 11, 'RGMAE'].str.replace('-', '').str.replace(' ', '')

  df.loc[df['GSPAI'] == 11, 'RGPAI'] = df.loc[df['GSPAI'] == 11, 'RGPAI'].str.replace('-', '').str.replace(' ', '')
  return df

In [39]:
def adiciona_colunas_codprod_codpai_cod_mae(df, genealogia_abcz):
  sub_tabela_gs_0 = df[df['gs'] == 0]
  genealogia_abcz = genealogia_abcz.rename(columns={'rgd2': 'RGVACA'})
  df_temporaria1 = pd.merge(sub_tabela_gs_0, genealogia_abcz[['RGVACA', 'cod_prod', 'cod_pai', 'cod_mae']], on=['RGVACA'], how='left', suffixes=('', '_drop'))
  sub_tabela_gs_0=df_temporaria1
  sub_tabela_gs_0 = sub_tabela_gs_0.drop_duplicates()

  df_temporaria4 = pd.merge(df, sub_tabela_gs_0[['RGVACA', 'cod_prod', 'cod_pai', 'cod_mae', 'IDANIMAL']], on=['RGVACA', 'IDANIMAL'], how='left', suffixes=('', '_drop'))
  df=df_temporaria4
  df = df.drop_duplicates()
  return df

In [40]:
def pegar_codpai_codmae_abcz_rg_3_letras(df, genealogia_abcz):

  genealogia_abcz = genealogia_abcz.rename(columns={'rgd2': 'RGVACA'})
  #Criando subtabela para maes
  subtabela_mae = df[df['GSMAE'] == 11]
  #e o cod_mae e nulo
  subtabela_mae
  # selecionar apenas as linhas que começam com letras
  subtabela_mae = subtabela_mae[subtabela_mae['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

  #Buscando cod_mae na abcz
  genealogia_abcz = genealogia_abcz.rename(columns={'cod_prod':'cod_prod_mae_gen'})
  subtabela_mae = pd.merge(subtabela_mae, genealogia_abcz[['RGVACA', 'cod_prod_mae_gen']], left_on=['RGMAE'], right_on=['RGVACA'], how='left', indicator=True, suffixes=('', '_drop'))
  subtabela_mae = subtabela_mae.drop_duplicates()
  subtabela_mae = subtabela_mae.drop(subtabela_mae.filter(regex='_drop$').columns, axis=1)
  subtabela_mae = subtabela_mae[subtabela_mae['_merge'] == 'both'].drop(columns='_merge')

  #Adicionando coluna cod_prod_mae no df
  df = pd.merge(df, subtabela_mae[['RGMAE', 'cod_prod_mae_gen']], left_on=['RGMAE'], right_on=['RGMAE'], how='left', suffixes=('', '_drop'))
  df = df.drop_duplicates()

  #Criando subtabela em que gspai = 11
  subtabela_pai = df[df['GSPAI'] == 11]

  # selecionar apenas as linhas que começam com letras
  subtabela_pai = subtabela_pai[subtabela_pai['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

  #Pegando cod_pai na ABCZ
  genealogia_abcz = genealogia_abcz.rename(columns={'cod_prod_mae_gen':'cod_prod_pai_gen'})
  subtabela_pai = pd.merge(subtabela_pai, genealogia_abcz[['RGVACA', 'cod_prod_pai_gen']], left_on=['RGPAI'], right_on=['RGVACA'], how='left', indicator=True, suffixes=('', '_drop'))
  subtabela_pai = subtabela_pai.drop_duplicates()
  subtabela_pai = subtabela_pai.drop(subtabela_pai.filter(regex='_drop$').columns, axis=1)
  subtabela_pai = subtabela_pai[subtabela_pai['_merge'] == 'both'].drop(columns='_merge')

  #Criando coluna do cod_prod_pai_gen no df
  df = pd.merge(df, subtabela_pai[['RGPAI', 'cod_prod_pai_gen']], left_on=['RGPAI'], right_on=['RGPAI'], how='left', suffixes=('', '_drop'))
  df = df.drop_duplicates()

  #Atualizando valores da cod_pai e cod_mae do df com os valores das colunas inseridas
  df['cod_pai'].fillna(df['cod_prod_pai_gen'], inplace=True)
  df['cod_mae'].fillna(df['cod_prod_mae_gen'], inplace=True)
  #Dando drop nas colunas antigas
  df = df.drop(columns='cod_prod_pai_gen')
  df = df.drop(columns='cod_prod_mae_gen')

  return df


In [41]:
def pegar_codpai_codmae_abcz_rg_diferente_3_letras(df, genealogia_abcz):
  genealogia_abcz = genealogia_abcz.rename(columns={'rgd2': 'RGVACA'})
  #Criando subtabela
  subtabela_pai = df[df['GSPAI'] == 11]
  # selecionar apenas as linhas que NÃO começam com letras
  subtabela_pai = subtabela_pai[~subtabela_pai['RGPAI'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

  #Pegando cod_pai na abcz
  genealogia_abcz = genealogia_abcz.rename(columns={'cod_prod':'cod_prod_pai_gen'})
  subtabela_pai = subtabela_pai.drop_duplicates()
  genealogia_abcz = genealogia_abcz.drop_duplicates()
  subtabela_pai['nome4caracteres'] = subtabela_pai['nomep'].str.slice(start=0, stop=4)
  genealogia_abcz['nome4caracteres'] = genealogia_abcz['nome'].str.slice(start=0, stop=4)
  subtabela_pai = pd.merge(subtabela_pai, genealogia_abcz[['RGVACA', 'nome4caracteres','cod_prod_pai_gen']], left_on=['RGPAI', 'nome4caracteres'], right_on=['RGVACA', 'nome4caracteres'], how='left', indicator=True, suffixes=('', '_drop'))
  subtabela_pai = subtabela_pai.drop_duplicates()
  subtabela_pai = subtabela_pai.drop(subtabela_pai.filter(regex='_drop$').columns, axis=1)
  subtabela_pai = subtabela_pai[subtabela_pai['_merge'] == 'both'].drop(columns='_merge')

  #Adicionando coluna de cod_pai_gen no df
  df = pd.merge(df, subtabela_pai[['RGPAI', 'cod_prod_pai_gen']], left_on=['RGPAI'], right_on=['RGPAI'], how='left', suffixes=('', '_drop'))
  df = df.drop_duplicates()

  #Criando subtabela mae
  genealogia_abcz = genealogia_abcz.rename(columns={'cod_prod':'cod_prod_mae_gen'})
  subtabela_mae = df[df['GSMAE'] == 11]
  # selecionar apenas as linhas que NÃO começam com letras
  subtabela_mae = subtabela_mae[~subtabela_mae['RGMAE'].apply(lambda x: bool(re.match(r'^[a-zA-Z]{3}', x)))]

  #Pegando cod_mae na abcz
  subtabela_mae['nome11caracteres'] = subtabela_mae['nomem'].str.slice(start=0, stop=11)
  genealogia_abcz = genealogia_abcz.rename(columns={'cod_prod_pai_gen':'cod_prod_mae_gen'})
  #criando coluna com os 10 caracteres do nome
  subtabela_mae = pd.merge(subtabela_mae, genealogia_abcz[['RGVACA','nome4caracteres' ,'cod_prod_mae_gen']], left_on=['RGMAE', 'nome11caracteres'], right_on=['RGVACA', 'nome4caracteres'], how='left', indicator=True, suffixes=('', '_drop'))
  subtabela_mae = subtabela_mae.drop_duplicates()
  subtabela_mae = subtabela_mae.drop(subtabela_mae.filter(regex='_drop$').columns, axis=1)
  subtabela_mae = subtabela_mae[subtabela_mae['_merge'] == 'both'].drop(columns='_merge')

  #Adicionando coluna de cod_mae_gen no df
  df = pd.merge(df, subtabela_mae[['RGMAE', 'cod_prod_mae_gen']], left_on=['RGMAE'], right_on=['RGMAE'], how='left', suffixes=('', '_drop'))
  df = df.drop_duplicates()

  #Atualizando cod_pai e cod_mae com base nas novas colunas
  df['cod_pai'].fillna(df['cod_prod_pai_gen'], inplace=True)
  df['cod_mae'].fillna(df['cod_prod_mae_gen'], inplace=True)
  #Dando drop nas colunas antigas
  df = df.drop(columns='cod_prod_pai_gen')
  df = df.drop(columns='cod_prod_mae_gen')
  return df

In [42]:
def adiciona_coluna_reb(df, criadores):
  criadores = criadores.rename(columns={'codgiro':'reb_giro'})
  df = pd.merge(df, criadores[['reb_giro', 'reb']], on=['reb_giro'], how='left')
  df = df.drop_duplicates()
  return df

In [43]:
def calcula_dias_de_diferenca_entre_partos(df, data_2022):
  df_diferenca_entre_partos_novos_e_do_banco = pd.merge(df, data_2022, on=['IDANIMAL'], suffixes=('', '_drop'))
  df_diferenca_entre_partos_novos_e_do_banco = df_diferenca_entre_partos_novos_e_do_banco.drop_duplicates()
  #df_diferenca_entre_partos_novos_e_do_banco = df_diferenca_entre_partos_novos_e_do_banco.drop(df_diferenca_entre_partos_novos_e_do_banco.filter(regex='_drop$').columns, axis=1)
  df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']] = df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']].astype(str)
  df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']] = df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop']].apply(pd.to_datetime)
  df_diferenca_entre_partos_novos_e_do_banco['diferenca_em_dias'] = (df_diferenca_entre_partos_novos_e_do_banco['parto'] - df_diferenca_entre_partos_novos_e_do_banco['parto_drop']).dt.days
  return df_diferenca_entre_partos_novos_e_do_banco
  #df_diferenca_entre_partos_novos_e_do_banco[['parto', 'parto_drop', 'diferenca_em_dias']]

In [44]:
def verifica_se_ha_id_animal_planilha_antiga(df, df_antigo):
  df_animais_com_ID = pd.merge(df, df_antigo, on =['IDANIMAL'], suffixes=('', '_drop'))
  df_animais_com_ID = df_animais_com_ID.drop(df_animais_com_ID.filter(regex='_drop$').columns, axis=1)
  df_animais_com_ID = df_animais_com_ID.drop_duplicates()
  return df_animais_com_ID

In [45]:
def gera_df_animais_sem_id_planilha_antiga(df, planilha_antiga):
  df_animais_sem_id = pd.merge(df, planilha_antiga, on='IDANIMAL', how='left', indicator=True, suffixes=('', '_drop'))
  df_animais_sem_id = df_animais_sem_id.drop(df_animais_sem_id.filter(regex='_drop$').columns, axis=1)
  df_animais_sem_id = df_animais_sem_id[df_animais_sem_id['_merge'] == 'left_only'].drop(columns='_merge')
  return df_animais_sem_id

In [46]:
def adiciona_id_produto_nos_animais_com_id(df, genealogia_embrapa):
  df_genealogia_embrapa['IDANIMAL'] = df_genealogia_embrapa['IDANIMAL'].astype('int')
  df = pd.merge(df, df_genealogia_embrapa[['IDANIMAL', 'id_produto']], on='IDANIMAL', how='left', indicator=True)
  df = df.drop_duplicates()
  df = df.drop(columns='_merge')
  return df

In [47]:
abre_conexao_banco_de_dados()
df_genealogia_embrapa = retorna_tabelas_banco_de_dados('genealogia_embrapa')
df_genealogia_embrapa =df_genealogia_embrapa.rename(columns={'idanimal': 'IDANIMAL'})
df_genealogia_abcz = retorna_tabelas_banco_de_dados('genealogia_abcz')
df_criadores = retorna_tabelas_banco_de_dados('criadores')
df_producao = retorna_tabelas_banco_de_dados('producao_embrapa')
df_fazendas = retorna_tabelas_banco_de_dados('fazendas')
pedigree = pd.read_csv("pedrigree - Girolando - 17-02-2023.txt", encoding='ISO-8859-1')
dados_2023 = pd.read_csv("PNMGL - Dados girolando - 17-02-23.txt", encoding='ISO-8859-1')
dados_2022 = pd.read_csv("PNMGL - Dados_old - 17-02-2023.txt", encoding='ISO-8859-1')

In [48]:
df_dados_novos = cria_df_com_somente_os_dados_novos(dados_2022, dados_2023)

In [49]:
df_dados_novos = adiciona_coluna_gs_aos_dados_novos(df_dados_novos, pedigree)

In [50]:
df_dados_novos = retira_acentos_espacos(df_dados_novos)

In [51]:
df_dados_novos =adiciona_colunas_codprod_codpai_cod_mae(df_dados_novos, df_genealogia_abcz)

In [52]:
df_dados_novos = pegar_codpai_codmae_abcz_rg_3_letras(df_dados_novos, df_genealogia_abcz)

In [53]:
df_dados_novos = pegar_codpai_codmae_abcz_rg_diferente_3_letras(df_dados_novos, df_genealogia_abcz)

In [54]:
df_dados_novos = adiciona_coluna_reb(df_dados_novos, df_criadores)

In [55]:
dif_partos = calcula_dias_de_diferenca_entre_partos(df_dados_novos, dados_2022)

In [56]:
df_animais_com_id = verifica_se_ha_id_animal_planilha_antiga(df_dados_novos, dados_2022)

In [57]:
df_animais_sem_id = gera_df_animais_sem_id_planilha_antiga(df_dados_novos, dados_2022)
df_animais_sem_id

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,rac,nordc,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,reb
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2,2,74612019122,NaN,5,F,NaN,NaN,NaN,1291
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2,2,106392021122,NaN,4,F,NaN,NaN,9513273.0,10639-GIRO
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2,2,60392020122,NaN,4,F,NaN,4320031.0,NaN,NaN
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,2,2,19452021222,NaN,4,F,NaN,NaN,10621711.0,37634-ABCZ
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2,2,731192021222,NaN,4,F,NaN,3540404.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1301,1945,JPZ QUIRERA SHAMROCK ASTER FIV,12,4032-Z,LADYS-MANOR PL SHAMROCK-ET,10,AX135714,JPZ ASTER FIV,11,JPZ64,...,2,2,19452022222,NaN,4,F,NaN,NaN,11428022.0,37634-ABCZ
1302,2381,DALIA I FIV BRADLEY JM NOVO HO,12,4906-AZ,ROCKALLI BRADLEY,10,AX117720,DALIA NOVO HORIZONTE,11,JMCH76,...,2,2,23812021222,NaN,4,F,NaN,NaN,12866148.0,31300-ABCZ
1305,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,2,2,1322019222,NaN,6,F,NaN,NaN,NaN,31835-ABCZ
1306,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,2,2,69152021222,NaN,5,F,NaN,NaN,NaN,27729-ABCZ


In [58]:
df_animais_com_id = adiciona_id_produto_nos_animais_com_id(df_animais_com_id, df_genealogia_embrapa)
df_animais_com_id

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,nordc,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,reb,id_produto
0,6029,DEMASIA FR RECREIO,14,0869-T,JAGUAR TE DO GAVIAO,11,GAV291,ERMELINDA MERGULHAO,12,AD-1407,...,2,60292021122,NaN,2,F,NaN,4176523.0,NaN,31365-ABCZ,GIRO1383626
1,7450,JACA JAYZ SJ LALU,34,6411-AH,HORNLAND JAYZ-ET,10,AX118150,GAYA FIV SANSAO SJ LALU,12,1898-P,...,2,74502020132,NaN,6,F,NaN,NaN,NaN,7450-GIRO,GIRO1608903
2,3304,CAITA OCIDENTE MJP DA CAPELA P,12,8634-X,OCIDENTE BEM FEITOR CAL,11,CAL4605,MARINA DA CAPELA PAIVA,10,A-3096,...,2,33042020122,NaN,4,F,NaN,3287666.0,NaN,3304-GIRO,GIRO1215331
3,5569,LONDRINA GENGIS KHAN RECANTO D,12,0021-AW,GENGIS KHAN DE BRASILIA,11,RRP6097,MEARA JUDITH GOLDWYN BELLA-TE,10,BX410396,...,2,55692021222,NaN,4,F,NaN,9265878.0,NaN,5569-GIRO,GIRO2022656
4,5569,BABALU METEORO DO MORRO,38,0148-AK,METEORO DE BRASILIA,11,B5226,SEGUNDA DO MORRO,34,7468-T,...,2,55692021222,NaN,3,F,NaN,2225848.0,NaN,5569-GIRO,GIRO1653303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,1129,MORADIA ANGICO,7,9556-F,VALE OURO DE BRASILIA,11,A6796,CABANA ANGICO,78,1606,...,2,11292021222,NaN,4,F,NaN,1351076.0,NaN,52721,GIRO1066128
416,2381,CLASSICA OCIDENTE JM NOVO HORI,14,5193-AE,OCIDENTE BEM FEITOR CAL,11,CAL4605,CINDERELA MODELO JM NOVO HORIZ,12,AF-2652,...,2,23812020122,NaN,2,F,NaN,3287666.0,NaN,31300-ABCZ,GIRO1571455
417,5569,GABI FIV VALE OURO EVD,14,4417-AT,VALE OURO TE SILVANIA,11,EFC464,LAMBADA DRAMATIC SANTA LUZIA,12,AG-0944,...,2,55692020122,NaN,2,F,NaN,3890526.0,NaN,5569-GIRO,GIRO1797713
418,6029,DUNCAN FR RECREIO,14,0928-T,JAGUAR TE DO GAVIAO,11,GAV291,NILOPOLIS FR RECREIO,12,AB-0524,...,2,60292021222,NaN,2,F,NaN,4176523.0,NaN,31365-ABCZ,GIRO1383629


In [59]:
def adiciona_animais_sem_id_que_possuem_id_na_gen_embrapa_ao_df_animais_com_id(animais_sem_id, df_animais_id, genealogia_embrapa):
  ids_encontrados = pd.merge(animais_sem_id, genealogia_embrapa['IDANIMAL'], left_on=['IDANIMAL'], right_on=['IDANIMAL'], how='left', suffixes=('', '_drop'), indicator=True)
  ids_encontrados = ids_encontrados.drop_duplicates()
  ids_encontrados = ids_encontrados.drop(ids_encontrados.filter(regex='_drop$').columns.difference(['IDANIMAL_drop']), axis=1)
  ids_encontrados = ids_encontrados[ids_encontrados['_merge'] == 'both']

  # dropa as colunas que não são necessárias
  ids_encontrados = ids_encontrados.drop(columns='_merge')
  df_animais_id = pd.concat([df_animais_id, ids_encontrados])
  df_animais_id = df_animais_id.drop_duplicates()
  return df_animais_id

In [60]:
df_animais_com_id = adiciona_animais_sem_id_que_possuem_id_na_gen_embrapa_ao_df_animais_com_id(df_animais_sem_id, df_animais_com_id, df_genealogia_embrapa)
df_animais_com_id

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,nordc,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,reb,id_produto
0,6029,DEMASIA FR RECREIO,14,0869-T,JAGUAR TE DO GAVIAO,11,GAV291,ERMELINDA MERGULHAO,12,AD-1407,...,2,60292021122,NaN,2,F,NaN,4176523.0,NaN,31365-ABCZ,GIRO1383626
1,7450,JACA JAYZ SJ LALU,34,6411-AH,HORNLAND JAYZ-ET,10,AX118150,GAYA FIV SANSAO SJ LALU,12,1898-P,...,2,74502020132,NaN,6,F,NaN,NaN,NaN,7450-GIRO,GIRO1608903
2,3304,CAITA OCIDENTE MJP DA CAPELA P,12,8634-X,OCIDENTE BEM FEITOR CAL,11,CAL4605,MARINA DA CAPELA PAIVA,10,A-3096,...,2,33042020122,NaN,4,F,NaN,3287666.0,NaN,3304-GIRO,GIRO1215331
3,5569,LONDRINA GENGIS KHAN RECANTO D,12,0021-AW,GENGIS KHAN DE BRASILIA,11,RRP6097,MEARA JUDITH GOLDWYN BELLA-TE,10,BX410396,...,2,55692021222,NaN,4,F,NaN,9265878.0,NaN,5569-GIRO,GIRO2022656
4,5569,BABALU METEORO DO MORRO,38,0148-AK,METEORO DE BRASILIA,11,B5226,SEGUNDA DO MORRO,34,7468-T,...,2,55692021222,NaN,3,F,NaN,2225848.0,NaN,5569-GIRO,GIRO1653303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,8689,JOA EMBAIXADOR LAGLORIA,14,6341-M,EMBAIXADOR LAGLORIA,11,MAFV20,FLORINDA LAGLORIA,12,AE-2426,...,2,86892019122,NaN,2,F,NaN,9392611.0,NaN,8689-GIRO,NaN
544,9373,JAB BARONEZA OPINION,78,8900-AI,ROYAL OPINION DECEMBER,10,AX119971,INTEGRALAT FIV 77761 KIRKLAND,34,AP-1389,...,2,93732020122,NaN,7,F,NaN,NaN,NaN,52706,NaN
563,5192,SUGESTIVA FIV DE BRAS.,11,RRP8637,JAGUAR TE DO GAVIAO,11,GAV291,JESSICA DE BRAS.,11,RRP6775,...,2,51922020132,NaN,0,F,16309606.0,4176523.0,11309453.0,5192-GIRO,NaN
569,6029,GRUTA FR RECREIO,12,4847-BG,0,0,0,0,0,0,...,2,60292021222,NaN,4,F,NaN,NaN,NaN,31365-ABCZ,NaN


In [61]:
def atualiza_animais_sem_id(animais_sem_id, df_animais_id):

  #Tirando o sufixo da tabela

  animais_sem_id = pd.merge(animais_sem_id, df_animais_id, on=['IDANIMAL'], how='left', indicator=True, suffixes=('', '_drop'))
  animais_sem_id = animais_sem_id[animais_sem_id['id_produto'].isnull()]
  animais_sem_id = animais_sem_id.drop(animais_sem_id.filter(regex='_drop$').columns, axis=1)
  # Seleciona apenas as linhas que possuem valores nulos na coluna "id_produto"

  animais_sem_id = animais_sem_id.drop(columns='_merge')
  return animais_sem_id

In [62]:
df_animais_sem_id = atualiza_animais_sem_id(df_animais_sem_id, df_animais_com_id)
df_animais_sem_id

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,nordc,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,reb,id_produto
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2,74612019122,NaN,5,F,NaN,NaN,NaN,1291,NaN
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2,106392021122,NaN,4,F,NaN,NaN,9513273.0,10639-GIRO,NaN
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2,60392020122,NaN,4,F,NaN,4320031.0,NaN,NaN,NaN
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,2,19452021222,NaN,4,F,NaN,NaN,10621711.0,37634-ABCZ,NaN
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2,731192021222,NaN,4,F,NaN,3540404.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,1945,JPZ QUIRERA SHAMROCK ASTER FIV,12,4032-Z,LADYS-MANOR PL SHAMROCK-ET,10,AX135714,JPZ ASTER FIV,11,JPZ64,...,2,19452022222,NaN,4,F,NaN,NaN,11428022.0,37634-ABCZ,NaN
620,2381,DALIA I FIV BRADLEY JM NOVO HO,12,4906-AZ,ROCKALLI BRADLEY,10,AX117720,DALIA NOVO HORIZONTE,11,JMCH76,...,2,23812021222,NaN,4,F,NaN,NaN,12866148.0,31300-ABCZ,NaN
621,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,2,1322019222,NaN,6,F,NaN,NaN,NaN,31835-ABCZ,NaN
622,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,2,69152021222,NaN,5,F,NaN,NaN,NaN,27729-ABCZ,NaN


In [63]:
df_animais_sem_id = df_animais_sem_id.drop_duplicates()
df_animais_sem_id

,reb_giro,nomea,COMPRAC,RGVACA,nomep,GSPAI,RGPAI,nomem,GSMAE,RGMAE,...,nordc,rebanoe,ordem,gs,sexo,cod_prod,cod_pai,cod_mae,reb,id_produto
0,7461,NANA IMPERADOR BAXTER RF SAO F,5858,6353-AH,IMPERADOR BAXTER VOLTA FRIA,58,1459,JURITI HERBY RF SAO FRANCISCO,58,6877-N,...,2,74612019122,NaN,5,F,NaN,NaN,NaN,1291,NaN
1,10639,LANAA FIV AFTERSHOCK BON.BELEM,12,9592-AG,MS ATLEES SHT AFTERSHOCK - ET,10,AX129928,JOANE FIV KUBERA,11,ACFG1840,...,2,106392021122,NaN,4,F,NaN,NaN,9513273.0,10639-GIRO,NaN
2,6039,EDILAINE IMPERIO FIV 4303 JARD,12,AADA0352,IMPERIO TE STA EDWIGES,11,RIG163,SERTEC QATAR FABIANA PROGRESS,10,HBB/B-291392,...,2,60392020122,NaN,4,F,NaN,4320031.0,NaN,NaN,NaN
3,1945,ARMA MERIDIAN DA MBL,12,1025-AN,SULLY HART MERIDIAN-ET,10,AX138576,FAMA,11,JNR167,...,2,19452021222,NaN,4,F,NaN,NaN,10621711.0,37634-ABCZ,NaN
4,73119,CELESTE 136 TEATRO FIV COOPERR,12,1239-BF,TEATRO DA SILVANIA,11,EFC383,LAGOS ASTRE QUIMICA 795 TE,10,BX335536,...,2,731192021222,NaN,4,F,NaN,3540404.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,1945,JPZ QUIRERA SHAMROCK ASTER FIV,12,4032-Z,LADYS-MANOR PL SHAMROCK-ET,10,AX135714,JPZ ASTER FIV,11,JPZ64,...,2,19452022222,NaN,4,F,NaN,NaN,11428022.0,37634-ABCZ,NaN
620,2381,DALIA I FIV BRADLEY JM NOVO HO,12,4906-AZ,ROCKALLI BRADLEY,10,AX117720,DALIA NOVO HORIZONTE,11,JMCH76,...,2,23812021222,NaN,4,F,NaN,NaN,12866148.0,31300-ABCZ,NaN
621,132,QUINTA I COLE AGM TRIUNFO,34,9243-AG,CO-OP TOYSTORY COLE-ET,10,AX129567,QUINTA FIV M,12,0473-O,...,2,1322019222,NaN,6,F,NaN,NaN,NaN,31835-ABCZ,NaN
622,6915,LIA DAS ARABIAS,5858,6125-BR,TURBO DAS ARABIAS,5858,0079,LILIKA DAS ARABIAS,58,4916-AW,...,2,69152021222,NaN,5,F,NaN,NaN,NaN,27729-ABCZ,NaN
